#### Means to plot with matplotlib in font size=11
#### maybe we want to add a sub plot at the bottom, pg.221 of the book
#### 

In [ ]:
import pandas as pd

#### make sure your .json file name matches what you have in your folder

In [ ]:
dbcamhd = pd.read_json('dbcamhd9_4.json', orient="records", lines=True)
dbcamhd.tail()

In [ ]:
print("Total files: %i" % len(dbcamhd))
print("Total frames: %i" % dbcamhd.frame_count.sum())

# Deployment is gonna be the easiest way to change the window of dates looked at.

In [ ]:
filenames = list(dbcamhd.filename[(dbcamhd.deployment == 5) & (dbcamhd.frame_count > 5000) & (dbcamhd.frame_count < 30000)])
filenames.sort()
filenames[0]

In [ ]:
len(filenames)

In [ ]:
frame_numbers = [3841, 3933, 4052, 4171, 4263, 4382]

In [ ]:
import pycamhd as camhd
import numpy as np
from dask import delayed
import dask.array as dsa

In [ ]:
delayed_frame_list = []
for filename in filenames:
    delayed_moov_atom = delayed(camhd.get_moov_atom)(filename)         
    for frame_number in frame_numbers:
        delayed_frame = delayed(camhd.get_frame)(filename, frame_number, 'gray16le', delayed_moov_atom)
        delayed_frame_list.append(dsa.from_delayed(delayed_frame, (1080, 1920), np.uint16))
delayed_frame_array = dsa.stack(delayed_frame_list)
delayed_frame_array

In [ ]:
len(delayed_frame_array)

## USE the output above to trim down the size of your array if you changed anything ([x by 6] where x by 6 is just less then or equal to your len(delayed_frame_array)

In [ ]:
frame = delayed_frame_array[1000*6].compute()
frame.shape

## Oh God, Why Is EVERYTHING dark

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.rc('figure', figsize=(11, 11))
fig, ax = plt.subplots()
im1 = ax.imshow(frame)
im1.set_cmap('gray')
plt.yticks(np.arange(0,1081,270))
plt.xticks(np.arange(0,1921,480))
rect = patches.Rectangle((10,10),1024,1024,linewidth=1.5,edgecolor='w',facecolor='none')
ax.add_patch(rect)
plt.show();

In [ ]:
def butterworth(d1, d2, n):
    x = np.arange(-1024/2+0.5,1024/2+1-0.5)
    xx, yy = np.meshgrid(x, x)
    d = np.sqrt(xx**2+yy**2)
    bff = (1 - (1./(1 + (d/d1)**(2*n))))*(1/(1 + (d/d2)**(2*n)))
    return bff

### The donut looks good

In [ ]:
d1 = 20 # low cut wavenumber
d2 = 400 # high cut wavenumber
n = 4
bff = butterworth(d1, d2, n)
plt.rc('figure', figsize=(6, 6))
imgplot = plt.imshow(bff, cmap='gray')

In [ ]:
def frame_filter(frame, d1, d2, n):
    if frame.ndim == 3 and frame.shape[0] == 1:
        I = np.squeeze(frame[0, 0:1024, 0:1024])
    else:
        I = frame[0:1024, 0:1024]
    bff = butterworth(d1, d2, n)
    I_fft = np.fft.fft2(I)
    I_fft_shift = np.fft.fftshift(I_fft)
    I_fft_shift_filt = I_fft_shift*bff # filter with the Butterworth filter
    I_fft_filt = np.fft.ifftshift(I_fft_shift_filt)
    I_filt = np.fft.ifft2(I_fft_filt)
    return I_filt

In [ ]:
def calc_floc_proxy(frame, d1, d2, n):
    I_filt = frame_filter(frame, d1, d2, n)
    return np.array([(np.absolute(I_filt)>4000).sum()])

In [ ]:
I_filt = frame_filter(frame, d1, d2, n)

## Fucking Abyss Again, This Means the Camera Frame Numbers Are Off?
#### Still we will push on

In [ ]:
plt.rc('figure', figsize=(6, 6))
imgplot = plt.imshow(np.absolute(I_filt)>4000, cmap='gray')
plt.title('floc_proxy value = %i' % (np.absolute(I_filt)>4000).sum());

In [ ]:
floc_proxy = dsa.map_blocks(calc_floc_proxy, delayed_frame_array, d1, d2, n, dtype='i8', drop_axis=[1,2])
floc_proxy

In [ ]:
%%time
results = floc_proxy[0:40].compute()

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

#### Workers, Cores, Memory=0 what is happening

In [ ]:
from dask.distributed import Client
client = Client(cluster)
client

In [ ]:
%%time
results = floc_proxy[0:400].compute()

In [ ]:
dbcamhd